In [ ]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
batch_size=32
size=64,64
train_datagen = ImageDataGenerator(validation_split=0.2) 

train_generator = train_datagen.flow_from_directory(
    "/content/drive/My Drive/Photos",
    target_size=(224,224),
    subset='training') 

validation_generator = train_datagen.flow_from_directory(
    "/content/drive/My Drive/Photos", 
    target_size=(224,224),
    subset='validation') 

Found 320 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [ ]:
from keras.models import Sequential, Model, load_model
from keras import applications
from keras import optimizers
from keras.layers import Dropout, Flatten, Dense
from keras.applications import resnet50
import tensorflow as tf
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy


img_rows, img_cols, img_channel = 224, 224, 3

from keras.applications import inception_v3
from keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(weights = 'imagenet', include_top = False, input_shape=(img_rows, img_cols, img_channel))

x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(512, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

model = Model(inputs = base_model.input, outputs = predictions)
for layer in base_model.layers:
    layer.trainable = False

model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:

model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_470 (Conv2D)             (None, 111, 111, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_470 (BatchN (None, 111, 111, 32) 96          conv2d_470[0][0]                 
__________________________________________________________________________________________________
activation_470 (Activation)     (None, 111, 111, 32) 0           batch_normalization_470[0][0]    
____________________________________________________________________________________________

In [ ]:

def init_callbacks():
  from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
  base_path = "/content/drive/My Drive/weights/Inception/"

 
  trained_models_path = base_path + 'model_weights'
  model_names = trained_models_path + '.{epoch:04d}--{val_loss:.4f}--{val_accuracy:.4f}.h5'
  model_checkpoint = ModelCheckpoint(model_names, monitor = 'val_accuracy', verbose=1,save_best_only=True)

  callbacks = [model_checkpoint]
  return callbacks

In [ ]:

hist = model.fit_generator(steps_per_epoch=train_generator.samples // batch_size, generator=train_generator, validation_data= validation_generator, validation_steps=validation_generator.samples // batch_size,epochs=25,callbacks= init_callbacks())

model.summary()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
10/10 [==============================] - 202s 19s/step - loss: 8.6612 - accuracy: 0.2515 - val_loss: 5.8373 - val_accuracy: 0.3438

Epoch 00001: val_accuracy improved from -inf to 0.34375, saving model to /content/drive/My Drive/weights/Inception/model_weights.0001--5.8373--0.3438.h5
Epoch 2/25
10/10 [==============================] - 87s 9s/step - loss: 4.7363 - accuracy: 0.3802 - val_loss: 3.6317 - val_accuracy: 0.4531

Epoch 00002: val_accuracy improved from 0.34375 to 0.45312, saving model to /content/drive/My Drive/weights/Inception/model_weights.0002--3.6317--0.4531.h5
Epoch 3/25
10/10 [==============================] - 85s 9s/step - loss: 3.0009 - accuracy: 0.4316 - val_loss: 2.5046 - val_accuracy: 0.4688

Epoch 00003: val_accuracy improved from 0.45312 to 0.46875, saving model to /content/drive/My Drive/weights/Inception/model_weights.0003--2.5046--0.4688.h5
Epoch 4/25
10/10 [==============================] - 76s 8s/step - loss: 2.4610 - accuracy: 0.4255 - val_loss: 